In [20]:
# Linkleri Ayarlama

import pandas as pd

df = pd.read_excel("25112024_Ulkeler w_iso3.xlsx")
df_cleaned = df.dropna(subset=['country_iso3'])
df_cleaned = df_cleaned.sort_values(by='country_iso3')
url_template = "https://widgets.weforum.org/future-of-jobs-2025/scorecards/score-card/economy/{}.json"
df_cleaned['URL'] = df_cleaned['country_iso3'].apply(lambda x: url_template.format(x))
df_cleaned.to_excel("Links.xlsx")

In [3]:
import os
import json
import pandas as pd

df = pd.read_excel("Links.xlsx")
df["Kontrol"] = df["Unnamed: 4"]
filtered_df = df[df["Kontrol"]=="Var"]
filtered_df = filtered_df[["country_code", "country_iso3", "Kontrol"]]
l = filtered_df['country_iso3'].tolist()
base_path = r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\Future Jobs\Future-Jobs\Datalar"
file_paths = [f"{base_path}\{country}.json" for country in l]

In [ ]:
def process_country_json(file_content):
    metrics = {
        'country_code': data['id'],
        'country_name': data['topSection']['title'],
        'working_age_population': data['topSection']['globalInfo']['value']
    }
    
    jobs_skills = data['sections'][0]['subSections'][0]['entries']
    metrics.update({
        'labour_market_churn': jobs_skills[0]['primaryPercentage'],
        'skill_disruption': jobs_skills[1]['primaryPercentage'],
        'dei_priorities': jobs_skills[2]['primaryPercentage'],
        'ai_exposure': jobs_skills[3]['primaryPercentage']
    })
    
    contextual = data['sections'][2]['subSections'][0]['entries']
    metrics.update({
        'labour_force_participation': contextual[0]['value'],
        'unemployment_rate': contextual[3]['value'],
        'secondary_education': contextual[6]['value'],
        'tertiary_education': contextual[7]['value']
    })
    
    workforce = data['sections'][5]['subSections'][0]['entries']
    metrics.update({
        'no_training_needed': workforce[0]['primaryPercentage'],
        'upskill_current_role': workforce[1]['primaryPercentage'],
        'upskill_redeploy': workforce[2]['primaryPercentage'],
        'unlikely_to_upskill': workforce[3]['primaryPercentage']
    })
    
    ai_strategy = data['sections'][-1]['subSections'][0]['entries']
    metrics.update({
        'ai_reskilling': ai_strategy[0]['primaryPercentage'],
        'ai_hiring': ai_strategy[1]['primaryPercentage'],
        'ai_job_transition': ai_strategy[2]['primaryPercentage']
    })
    
    return metrics


In [38]:
with open(file_paths[0], 'r', encoding='utf-8') as f:
    content = f.read()
data = json.loads(content) if isinstance(content, str) else content

import pandas as pd
import json

metrics = {
    'country_code': data['id'],
    'country_name': data['topSection']['title'],
    'working_age_population': data['topSection']['globalInfo']['value']
}

try:
    jobs_skills = data['sections'][0]['subSections'][0]['entries']
    metrics.update({
        'labour_market_churn': jobs_skills[0].get('primaryPercentage', None),
        'skill_disruption': jobs_skills[1].get('primaryPercentage', None),
        'dei_priorities': jobs_skills[2].get('primaryPercentage', None),
        'ai_exposure': jobs_skills[3].get('primaryPercentage', None)
    })
except (IndexError, KeyError):
    print("Jobs and Skills verisi çekilemedi")

try:
    contextual = data['sections'][2]['subSections'][0]['entries']
    metrics.update({
        'labour_force_participation': contextual[0].get('value', None),
        'unemployment_rate': contextual[3].get('value', None),
        'secondary_education': contextual[6].get('value', None),
        'tertiary_education': contextual[7].get('value', None)
    })
except (IndexError, KeyError):
    print("Contextual indicators verisi çekilemedi")

try:
    workforce = data['sections'][5]['subSections'][0]['entries']
    metrics.update({
        'no_training_needed': workforce[0].get('primaryPercentage', None),
        'upskill_current_role': workforce[1].get('primaryPercentage', None),
        'upskill_redeploy': workforce[2].get('primaryPercentage', None),
        'unlikely_to_upskill': workforce[3].get('primaryPercentage', None)
    })
except (IndexError, KeyError):
    print("Workforce training verisi çekilemedi")

try:
    ai_strategy = data['sections'][-1]['subSections'][0]['entries']
    metrics.update({
        'ai_reskilling': ai_strategy[0].get('primaryPercentage', None),
        'ai_hiring': ai_strategy[1].get('primaryPercentage', None),
        'ai_job_transition': ai_strategy[2].get('primaryPercentage', None)
    })
except (IndexError, KeyError):
    print("AI strategy verisi çekilemedi")

df = pd.DataFrame([metrics])

df.to_excel('country_metrics.xlsx', index=False)

In [47]:
with open(file_paths[0], 'r', encoding='utf-8') as f:
    content = f.read()
data = json.loads(content) if isinstance(content, str) else content

country = data["topSection"]["title"]

pivot_data = {}

for section in data["sections"]:
    section_title = section["title"]
    for sub_section in section["subSections"]:
        sub_section_title = sub_section["title"]
        for entry in sub_section.get("entries", []):
            entry_label = entry.get("label", sub_section_title)
            if entry_label not in pivot_data:
                pivot_data[entry_label] = {}
            pivot_data[entry_label][country] = entry.get("primaryValue", entry.get("value", ""))

pivot_df = pd.DataFrame.from_dict(pivot_data, orient='index')

pivot_df.index.name = "Category"
pivot_df.reset_index(inplace=True)

pivot_file_path = "data/ARE_pivot_data.xlsx"
pivot_df.to_excel("a.xlsx")